# Pulsar detection 

### Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

A pulsar is a celestial body which emits precise amounts of electromagnetic radiation. It is highly sought after by astrophysicists as it is the only place where they can observe matter indirectly at nuclear density. 



### Clearly state the question you will try to answer with your project


Can we build an accurate classifier to distinguish pulsars from other celestial bodies?



### Identify and describe the dataset that will be used to answer the question

The utilized dataset is [HTRU2 Data Set](https://archive.ics.uci.edu/ml/datasets/HTRU2) collected by Dr Robert Lyon, University of Manchester. 